In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
df = pd.read_dict('articles.csv')

AttributeError: module 'pandas' has no attribute 'read_dict'

In [4]:
df = pd.read_json('articles.json')

ValueError: Trailing data

In [1]:
#Launches Mongo Client
from pymongo import MongoClient
client = MongoClient()
biased_news = client.events.biased_news

In [2]:
client.database_names()

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused

In [4]:
#Launches Mongo Client
from pymongo import MongoClient

client = MongoClient()
biased_news = client.events.biased_news

In [5]:
#See where in mongo to place the articles
client.database_names()

['admin', 'catalog', 'companies', 'config', 'events', 'local']

In [6]:
#Create news bias events
db = client.events
biased_news = db.biased_news

In [8]:
#Sanity check
biased_news.count()

5719

In [10]:
list(biased_news.aggregate([{'$group' : {'_id': '$title', 'count': {'$sum': 1}}},
    {'$match': {'count': {'$gte': 2}}},
    ]))

[{'_id': '\r\n\t\t\tIsraeli Company Is Selling Autonomous Surveillance Systems to Governments Around the World\t\t',
  'count': 2},
 {'_id': '\r\n\t\t\tCommercial for Asthma Medication Features Unsafe Use of Technology.  Respiratory Health Is Affected by Cell Phone and WiFi Radiation and Electrical Pollution (Electrosmog).\t\t',
  'count': 2},
 {'_id': '\r\n\t\t\tMIT Creates Serial Killer A.I. Personality With Reddit Experiment\t\t',
  'count': 2},
 {'_id': '\r\n\t\t\tSomething Unprecedented Is Happening at Bilderberg 2018\t\t',
  'count': 2},
 {'_id': '', 'count': 2},
 {'_id': '\t\t\t\t\t\t\tJudicial Nominations Update\t\t\t\t\t', 'count': 3},
 {'_id': '\t\t\t\t\t\t\tMore Twisted Logic on Obamacare\t\t\t\t\t',
  'count': 2},
 {'_id': '\t\t\t\t\t\t\tTrump’s Trade War Sparks Congressional Backlash\t\t\t\t\t',
  'count': 2},
 {'_id': '\t\t\t\t\t\t\tPoll: Narrow Plurality of Voters Says Mueller Probe ‘Legitimate’\t\t\t\t\t',
  'count': 2},
 {'_id': '\t\t\t\t\t\t\tMueller Files New Indictm

In [11]:
#Create cursor
cursor = biased_news.find()

In [14]:
#Loads from Mongo
biased_news_df = pd.DataFrame(list(cursor))

In [20]:
biased_news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5719 entries, 0 to 5718
Data columns (total 6 columns):
_id           5719 non-null object
body          5719 non-null object
num_source    4354 non-null float64
source        4354 non-null object
sub_title     1365 non-null object
title         5719 non-null object
dtypes: float64(1), object(5)
memory usage: 268.2+ KB


In [64]:
biased_news_df.source.value_counts()

Mother Jones       912
Huffington Post    640
Info wars          538
Reuters            520
Breitbart          502
National Review    430
Activist Post      427
Alabama Today      385
Name: source, dtype: int64

In [23]:
beast_list_o_articles = pickle.load(open("pickle_files/beast_list_o_articles.pkl", "rb" ))
#mj_more_articles = pickle.load(open("pickle_files/beast_list_o_articles.pkl", "rb" ))

In [24]:
beast_list_o_articles

549

In [25]:
beast_df = pd.DataFrame.from_dict(beast_list_o_articles)

In [27]:
beast_df['num_source'] = 3
beast_df['source'] = 'Daily Beast'

In [48]:
gr_list_o_articles = pickle.load(open("pickle_files/gr_list_o_articles.pkl", "rb" ))


In [49]:
len(gr_list_o_articles)

392

In [50]:
gr_df = pd.DataFrame.from_dict(gr_list_o_articles)

In [52]:
gr_df['num_source'] = 5
gr_df['source'] = 'Global Research'

In [54]:
ap_list_o_articles = pickle.load(open("pickle_files/ap_list_o_articles.pkl", "rb" ))


In [55]:
ap_df = pd.DataFrame.from_dict(ap_list_o_articles)

In [57]:
ap_df['num_source'] = 6
ap_df['source'] = 'Associate Press'

In [59]:
fox_list_o_articles = pickle.load(open("pickle_files/fox_list_o_articles.pkl", "rb" ))


In [61]:
fox_df = pd.DataFrame.from_dict(fox_list_o_articles)

In [62]:
fox_df['num_source'] = 7
fox_df['source'] = 'Fox News'

In [65]:
true_df = pd.concat([biased_news_df, beast_df, gr_df, ap_df, fox_df])

In [66]:
del true_df['sub_title']

In [67]:
del true_df['_id']

In [70]:
true_df = true_df.dropna(how='any')

In [71]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5579 entries, 0 to 234
Data columns (total 4 columns):
body          5579 non-null object
num_source    5579 non-null float64
source        5579 non-null object
title         5579 non-null object
dtypes: float64(1), object(3)
memory usage: 217.9+ KB


In [200]:
true_df.title.value_counts()

Do You Know What's Going On With Scott Pruitt's Aides?                                                                                                                                     1
Bernie Sanders political group sees 2nd departure, sparking concerns about racial tensions                                                                                                 1
\t\t\t\t\t\t\tBroad Enough to Matter\t\t\t\t\t                                                                                                                                             1
Jeff Sessions Uses Exceptional Power Over Immigration Courts To Limit Judges’ Authority                                                                                                    1
\n\t\t\tKay Ivey stands up for Mobile teacher who was to change ‘just pray’ t-shirt\t\t\t                                                                                                  1
\r\n\t\t\tIsrael Won’t Cooperate With UN Probe on Gaza 

In [198]:
#See what cases the article appears in
huh = true_df['title'] == "We Need Your Voice: Independent Critical Analysis In Danger"
wha = true_df[huh]
wha.index

Int64Index([5], dtype='int64')

In [175]:
#Drop the redundant cases and repeate step 1
label = wha.index
true_df = true_df.drop(labels=label[1:],axis=0)

In [180]:
kill = true_df.title[eliminate]

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

In [184]:
eliminate = true_df.title.value_counts()[true_df.title.value_counts() > 1]


In [199]:
for title in eliminate.index:
    huh = true_df['title'] == title
    wha = true_df[huh]
    label = wha.index
    true_df = true_df.drop(labels=label[1:],axis=0)

In [201]:
true_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4092 entries, 0 to 234
Data columns (total 4 columns):
body          4092 non-null object
num_source    4092 non-null float64
source        4092 non-null object
title         4092 non-null object
dtypes: float64(1), object(3)
memory usage: 159.8+ KB


In [6]:
true_df = pickle.load(open("Project5/pickle_files/true_df.pkl", "rb" ))

FileNotFoundError: [Errno 2] No such file or directory: 'Project5/pickle_files/true_df.pkl'